In [ ]:
from pydantic import BaseModel
import pandas as pd

df = pd.read_excel("sample_data/SRO658Markup.xlsx", engine="openpyxl")



In [1]:
from openpyxl import load_workbook
import openpyxl

workbook = load_workbook("sample_data/SRO658Markup.xlsx", rich_text=True)

# Always working with the first sheet
worksheet = workbook.worksheets[0]

# Iterate the row (skip the header row)
for row in worksheet.iter_rows(min_row=2): # 1 = header, 2 = first data row (skipping the header row)
    for cell in row:
        # Skip empty cell or non-string cells
        if cell.value is None or not isinstance(cell.value, str):
            continue # continue to next iteration - doesn't proceed with next if-condition
        
        print("pass first if")

        # If entire cell is strike-through, clear the cell
        if cell.font and cell.font.strike:
            cell.value = None
            continue # continue to next interation - doesn't proceed with next if-condition
        
        print("pass second if")

        # Otherwise, at run-level formatting
        if cell.rich_text:
            kept_chunks = [
                run.text
                for run in cell.rich_text
                if not (run.font and run.font.strike) # Keep only non-strike runs
            ]
            cell.value = "".join(kept_chunks)
            cell.rich_text = None
            print("end of getattr")

workbook.save("sample_data/output_markup.xlsx")
        



pass first if
pass second if


AttributeError: 'Cell' object has no attribute 'rich_text'

In [2]:
import openpyxl, sys, pathlib, platform
print("openpyxl version  →", openpyxl.__version__)
print("module location   →", pathlib.Path(openpyxl.__file__).as_posix())
print("python executable →", sys.executable)
print("python version    →", platform.python_version())


openpyxl version  → 3.1.5
module location   → c:/Users/Tom/Documents/cross-checker/venv/Lib/site-packages/openpyxl/__init__.py
python executable → c:\Users\Tom\Documents\cross-checker\venv\Scripts\python.exe
python version    → 3.13.2


In [ ]:
from pydantic import BaseModel
import pandas as pd

df = pd.read_excel("sample_data/SRO658_DOORS.xlsx", engine="openpyxl")

df.columns
# df['Object Text']

In [ ]:
from pydantic import BaseModel
import pandas as pd

# Read and load the markup into dataframe.
df_markup = pd.read_excel("sample_data/SRO658Markup.xlsx", engine="openpyxl")
# Read and load the export from DOORS into dataframe.
df_doors = pd.read_excel("sample_data/SRO658_DOORS.xlsx", engine="openpyxl")

df_compare = df_markup['Object Text'].compare(df_doors['Object Text'], align_axis="index")
df_compare.infer_objects
# print(df_compare['self'])
# print(df_compare['other'])
